In [1]:
import data as da

In [2]:
""" 读取数据集并打乱并切分为训练集和测试集，
使用z-socre进行归一化处理，
使用PCA降维升维进行噪音处理"""
data_pro=da.Data()
data=data_pro.read_data()
xs,ys=data_pro.split_x_y(data)
xs_trans, xs_pca = data_pro.data_noise(10,xs)
xs, xs_train, xs_test, ys_train, ys_test = data_pro.data_processing(xs_pca,ys)

print('training set shape: ',xs_train.shape)
print('test set shape:     ', xs_test.shape)

确保inequality-white.csv在同一文件夹
training set shape:  (3918, 11)
test set shape:      (980, 11)


In [12]:
# 2 mlp
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

param_grid = [{
    'hidden_layer_sizes': [(8,2),(8,3),(8,4),(8)],
    'alpha':[0.0001,0.001,0.01,0.1,1,10,100],
    'learning_rate':['invscaling','constant']
}]

# hideen layer 采用（输入+输出）*2/3，参考https://stackoverflow.com/questions/10565868/multi-layer-perceptron-mlp-architecture-criteria-for-choosing-number-of-hidde
mpl_reg = MLPRegressor()
mpl_search = GridSearchCV(mpl_reg, param_grid,n_jobs=-1)
mpl_search.fit(xs_train, ys_train)

print("The best parameters is",mpl_search.best_params_)
print('score of parameter is',mpl_search.best_score_)

The best parameters is {'alpha': 0.01, 'hidden_layer_sizes': (8, 4), 'learning_rate': 'invscaling'}
score of parameter is 0.321032175640717


D:\Anaconda\envs\machine\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
from sklearn.neural_network import MLPRegressor

mpl_reg = MLPRegressor(hidden_layer_sizes=(8, 4), alpha=0.01, learning_rate='invscaling')

mpl_reg.fit(xs_train, ys_train)

ys_train_pred = mpl_reg.predict(xs_train)
ys_test_pred = mpl_reg.predict(xs_test)

import fit_result
fit_result.score(ys_train, ys_train_pred,ys_test, ys_test_pred)

Train score of kNN trian set 0.3390181821109304
Test score of kNN test set 0.3099347245952391 

Train mean error of kNN trian set 0.5470493057662941
Test mean error of kNN test set 0.6088465122003457 

Train R2 of kNN trian set 0.3387640565781517
Test R2 error of kNN test set 0.302430996508374


D:\Anaconda\envs\machine\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
from sklearn.model_selection import cross_validate
mpl_result=cross_validate(mpl_reg,xs,ys,cv=10,n_jobs=-1,scoring=('explained_variance','r2'))

print('time: ',mpl_result['fit_time'].mean())
print('variance score: ',mpl_result['test_explained_variance'].mean())
print('r2: ',mpl_result['test_r2'].mean(),'\n')

knn time:  3.636304998397827
knn variance score:  0.2052148748107129
knn r2:  -1.025865635065205 

